In [15]:
%matplotlib widget
%matplotlib widget
import os
from pathlib import Path
import time
import torch
import numpy as np
import math
import gc
from functools import partial
from dataset_new_v2 import Dataset, load_dataframes_from_folder, reverse_normalization
from torch.utils.data import DataLoader
from transformer_zerostep_new_v2 import GPTConfig, GPT, warmup_cosine_lr
import argparse
import warnings
import matplotlib.pyplot as plt
import glob
import pandas as pd

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams['axes.labelsize']=14
plt.rcParams['xtick.labelsize']=11
plt.rcParams['ytick.labelsize']=11
plt.rcParams['axes.grid']=True
plt.rcParams['axes.xmargin']=0

In [16]:
# Overall settings
out_dir = "out"
batch_size = 1024

model_name = "ckpt_50pct_recursive_h10_real_val_speed_correction_low_speed_v2.pt"

folder_path = '../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/'
folder_path_final = folder_path + "final/"
folder_path_final_fixed = folder_path + "final_fixed/"
# folder_path = '../../../in-context-bldc-data/simulated/50_percent_with_alfa_beta_speed_corrected'

# Compute settings
cuda_device = "cuda:0"
no_cuda = False
threads = 10
compile = False

# Configure compute
torch.set_num_threads(threads) 
use_cuda = not no_cuda and torch.cuda.is_available()
device_name  = cuda_device if use_cuda else "cpu"
device = torch.device(device_name)
device_type = 'cuda' if 'cuda' in device_name else 'cpu' # for later use in torch.autocast
torch.set_float32_matmul_precision("high")
print(torch.cuda.is_available())
# Create out dir
out_dir = Path(out_dir)
exp_data = torch.load(out_dir/model_name, map_location=device, weights_only=False)
seq_len = exp_data["cfg"].seq_len
nx = exp_data["cfg"].nx
exp_data["iter_num"]
print(seq_len)
print(exp_data["iter_num"])
print(exp_data['best_val_loss'])
print(exp_data["cfg"])
print(exp_data["cfg"].lr)

True
10
4931
0.00044166864245198667
Namespace(model_dir='out', out_file='ckpt_50pct_recursive_h10_real_val_speed_correction_low_speed_v2', in_file='ckpt_50pct_recursive_h10_real_val_speed_correction_low_speed_v2', init_from='scratch', seed=42, log_wandb=False, nx=4, nu=5, ny=1, seq_len=10, mag_range=(0.5, 0.97), phase_range=(0.0, 1.5707963267948966), fixed_system=False, n_layer=8, n_head=4, n_embd=16, dropout=0, bias=False, batch_size=128, max_iters=8000, warmup_iters=5000, lr=1e-05, weight_decay=0.0, eval_interval=10, eval_iters=10, fixed_lr=False, threads=16, no_cuda=False, cuda_device='cuda:0', compile=False, beta1=0.9, beta2=0.95, block_size=10, lr_decay_iters=8000, min_lr=1.0000000000000002e-06, decay_lr=True, eval_batch_size=128)
1e-05


In [17]:
model_args = exp_data["model_args"]
gptconf = GPTConfig(**model_args)
model = GPT(gptconf).to(device)


state_dict = exp_data["model"]
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    if k.startswith('module.'):
        state_dict[k[7:]] = v
        state_dict.pop(k)

model.load_state_dict(state_dict)
H = exp_data["cfg"].seq_len



number of parameters: 0.03M


In [21]:
folder_list = glob.glob(os.path.join(folder_path_final, "*"))

for file_idx in range(len(folder_list)):
# for file_idx in range(1):
    csv_list = glob.glob(os.path.join(folder_list[file_idx], "*.csv"))

    dfs = load_dataframes_from_folder(folder_list[file_idx])
    print(f"Loaded {len(dfs)} DataFrames from {folder_list[file_idx]}.") 
    dataset_exp = Dataset(dfs=dfs, seq_len=H)
    dataloader = DataLoader(dataset_exp, batch_size=batch_size)

    df_len = len(dfs)
    # df_len = 1
    rmse = np.zeros(df_len)
    u_full_all = []
    y_full_all = []
    # y_pred_all = []
    for i in range(df_len):
        u_full, y_full = dataset_exp.get_full_experiment(i)
        u_full, y_full = u_full.to(device), y_full.to(device)
        u_full_all.append(u_full)
        y_full_all.append(y_full)

    u_full_all = torch.stack(u_full_all, dim=0)
    y_full_all = torch.stack(y_full_all, dim=0)
    y_pred_all = torch.zeros_like(y_full_all)
    last_omega = torch.zeros((df_len, H, 1))  

    with torch.no_grad():
        for j in range(y_full_all.shape[1]):
            
            # at time instant j
            if j < H:
                input_val = u_full_all[:, :j+1, :]
                input_val[:, :j+1, 4] = last_omega[:,-j-1:,0]
                pred = model(input_val)[:,-1,:]
            else:
                input_val = u_full_all[:,j-H+1:j+1, :]
                input_val[:,:, 4] = last_omega[:,:,0]
                pred = model(input_val)[:,-1,:]
            
            y_pred_all[:,j,0] = pred[:,0]
            last_omega = torch.roll(last_omega, -1, 1)
            last_omega[:,-1,0] = y_pred_all[:,j,0]
        
        u_full_all, y_full_all, y_pred_all  = reverse_normalization(u_full_all, y_full_all, y_pred_all)
        
    for i in range(df_len):
        y_tmp = y_full_all[i,:,:].cpu().numpy()
        y_pred_tmp = y_pred_all[i,:,:].cpu().numpy()

        df = pd.read_csv(csv_list[i])
        df["omega_meta"] = y_pred_tmp
        df.to_csv(csv_list[i])
        # print(y_tmp)
        # print(y_pred_tmp)
        rmse[i] = np.sqrt(((y_tmp-y_pred_tmp)**2).mean())

    print("Average rmse: ", rmse.mean())

    # for i in range(1):

    #     # y_full_np = y_full_all.cpu().numpy()[:,:200,:]
    #     # y_pred_np = y_pred_all.cpu().numpy()[:,:200,:]
    #     # u_full_np = u_full_all.cpu().numpy()[:,:200,:]
    #     y_full_np = y_full_all.cpu().numpy()
    #     y_pred_np = y_pred_all.cpu().numpy()
    #     u_full_np = u_full_all.cpu().numpy()

    #     print(y_full_np.shape)
    #     print(u_full_np.shape)
    #     fig = plt.figure(figsize=(10,4))
    #     ax1 = fig.add_subplot(3,1,1)
    #     ax1.plot(y_full_np[i])
    #     ax1.plot(y_pred_np[i])
    #     ax2 = fig.add_subplot(3,1,2)
    #     ax2.plot(u_full_np[i,:,0])
    #     ax2.plot(u_full_np[i,:,1])
    #     ax3 = fig.add_subplot(3,1,3)
    #     ax3.plot(u_full_np[i,:,2])
    #     ax3.plot(u_full_np[i,:,3])


    # plt.show()










Loaded 15 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final\inertia05_ki-0.0029-kp-3.0000.
Average rmse:  34.81443557739258
Loaded 15 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final\inertia07_ki-0.0029-kp-3.0000.
Average rmse:  44.525853093465166
Loaded 15 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final\inertia09_ki-0.0029-kp-3.0000.
Average rmse:  40.765188217163086
Loaded 15 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final\inertia11_ki-0.0029-kp-3.0000.
Average rmse:  43.16288134256999
Loaded 15 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final\inertia13_ki-0.0029-kp-3.0000.
Average rmse:  34.61573053995768
Loaded 15 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final\inertia15_ki-0.0029-kp-3.0000.
Average rmse:  39.89146054585775


In [22]:
folder_list = glob.glob(os.path.join(folder_path_final_fixed, "*"))

for file_idx in range(len(folder_list)):
# for file_idx in range(1):
    csv_list = glob.glob(os.path.join(folder_list[file_idx], "*.csv"))

    dfs = load_dataframes_from_folder(folder_list[file_idx])
    print(f"Loaded {len(dfs)} DataFrames from {folder_list[file_idx]}.") 
    dataset_exp = Dataset(dfs=dfs, seq_len=H)
    dataloader = DataLoader(dataset_exp, batch_size=batch_size)

    df_len = len(dfs)
    # df_len = 1
    rmse = np.zeros(df_len)
    u_full_all = []
    y_full_all = []
    # y_pred_all = []
    for i in range(df_len):
        u_full, y_full = dataset_exp.get_full_experiment(i)
        u_full, y_full = u_full.to(device), y_full.to(device)
        u_full_all.append(u_full)
        y_full_all.append(y_full)

    u_full_all = torch.stack(u_full_all, dim=0)
    y_full_all = torch.stack(y_full_all, dim=0)
    y_pred_all = torch.zeros_like(y_full_all)
    last_omega = torch.zeros((df_len, H, 1))  

    with torch.no_grad():
        for j in range(y_full_all.shape[1]):
            
            # at time instant j
            if j < H:
                input_val = u_full_all[:, :j+1, :]
                input_val[:, :j+1, 4] = last_omega[:,-j-1:,0]
                pred = model(input_val)[:,-1,:]
            else:
                input_val = u_full_all[:,j-H+1:j+1, :]
                input_val[:,:, 4] = last_omega[:,:,0]
                pred = model(input_val)[:,-1,:]
            
            y_pred_all[:,j,0] = pred[:,0]
            last_omega = torch.roll(last_omega, -1, 1)
            last_omega[:,-1,0] = y_pred_all[:,j,0]
        
        u_full_all, y_full_all, y_pred_all  = reverse_normalization(u_full_all, y_full_all, y_pred_all)
        
    for i in range(df_len):
        y_tmp = y_full_all[i,:,:].cpu().numpy()
        y_pred_tmp = y_pred_all[i,:,:].cpu().numpy()

        df = pd.read_csv(csv_list[i])
        df["omega_meta"] = y_pred_tmp
        df.to_csv(csv_list[i])
        # print(y_tmp)
        # print(y_pred_tmp)
        rmse[i] = np.sqrt(((y_tmp-y_pred_tmp)**2).mean())

    print("Average rmse: ", rmse.mean())









Loaded 1 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\inertia05_ki-0.0029-kp-3.0000.
Average rmse:  67.40986633300781
Loaded 1 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\inertia07_ki-0.0029-kp-3.0000.
Average rmse:  74.98125457763672
Loaded 1 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\inertia09_ki-0.0029-kp-3.0000.
Average rmse:  69.17872619628906
Loaded 1 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\inertia11_ki-0.0029-kp-3.0000.
Average rmse:  69.72197723388672
Loaded 1 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\inertia13_ki-0.0029-kp-3.0000.
Average rmse:  63.02893829345703
Loaded 1 DataFrames from ../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\inertia15_ki-0.0029-kp-3.0000.
Average rmse:  56.59474182128906
